# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [29]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [30]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,atherton,53.5237,-2.4935,5.58,5.58,6,0.45,GB,1706649230
1,1,albany,42.6001,-73.9662,0.29,0.29,100,0.45,US,1706649008
2,2,arraial do cabo,-22.9661,-42.0278,26.92,26.92,75,3.60,BR,1706649230
3,3,whitehorse,60.7161,-135.0538,2.95,2.95,63,7.15,CA,1706649231
4,4,port elizabeth,-33.9180,25.5701,22.40,22.40,0,1.03,ZA,1706649231


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [31]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

map_Plot = city_data_df.dtypes

# Display the map
# YOUR CODE HERE

map_Plot


City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity      float64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [32]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE

ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"] < 25) & (city_data_df["Max Temp"] > 22) & (city_data_df["Cloudiness"] < 2)
& (city_data_df["Wind Speed"] < 4)]

# Drop any rows with null values
# YOUR CODE HERE
ideal_weather_df.dropna()

# Display sample data
# YOUR CODE HERE
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,port elizabeth,-33.9180,25.5701,22.40,22.40,0,1.03,ZA,1706649231
19,19,lompoc,34.6391,-120.4579,23.90,23.90,0,1.54,US,1706649233
82,82,giddings,30.1827,-96.9364,23.49,23.49,0,0.00,US,1706649242
224,224,valdivia,-39.8142,-73.2459,23.23,23.23,0,3.57,CL,1706649266
312,312,laguna,38.4210,-121.4238,23.43,23.43,0,3.09,US,1706649205
508,508,motueka,-41.1333,173.0167,22.28,22.28,0,0.58,NZ,1706649315
518,518,abu dhabi,24.4667,54.3667,24.41,24.41,0,0.51,AE,1706649317


### Step 3: Create a new DataFrame called `hotel_df`.

In [33]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = ""

# Display sample data
# YOUR CODE HERE
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
4,port elizabeth,ZA,-33.9180,25.5701,22.40,
19,lompoc,US,34.6391,-120.4579,23.90,
82,giddings,US,30.1827,-96.9364,23.49,
224,valdivia,CL,-39.8142,-73.2459,23.23,
312,laguna,US,38.4210,-121.4238,23.43,
508,motueka,NZ,-41.1333,173.0167,22.28,
518,abu dhabi,AE,24.4667,54.3667,24.41,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [34]:
# Set parameters to search for a hotel
radius = 10000 
params = {"categories": "accommodation.hotel", "apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"  # YOUR CODE HERE
    params["bias"] = f"proximity:{longitude},{latitude}"  # YOUR CODE HERE
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params) 
    
    # Convert the API response to JSON format
    name_address =  name_address.json() 
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port elizabeth - nearest hotel: No hotel found
lompoc - nearest hotel: No hotel found
giddings - nearest hotel: No hotel found
valdivia - nearest hotel: No hotel found
laguna - nearest hotel: No hotel found
motueka - nearest hotel: No hotel found
abu dhabi - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
4,port elizabeth,ZA,-33.9180,25.5701,22.40,No hotel found
19,lompoc,US,34.6391,-120.4579,23.90,No hotel found
82,giddings,US,30.1827,-96.9364,23.49,No hotel found
224,valdivia,CL,-39.8142,-73.2459,23.23,No hotel found
312,laguna,US,38.4210,-121.4238,23.43,No hotel found
508,motueka,NZ,-41.1333,173.0167,22.28,No hotel found
518,abu dhabi,AE,24.4667,54.3667,24.41,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [35]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE